In [1]:
import os
import numpy as np
from osgeo import gdal, gdal_array, gdalconst, ogr, osr
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
os.chdir(r"C:\Model\Clip")
from check_raster_files import *
from paths import *
from vector_geometry import *

In [2]:
folder_base = r"C:\Objects"
folder_in = os.path.join(folder_base, "Raw")
folder_out = os.path.join(folder_base, "Projected")
folder_bound = os.path.join(folder_base, "Bound")
folder_fishnet = os.path.join(folder_base, "Fishnet")
folder_clipped = os.path.join(folder_base, "Clipped")
folder_cropped = os.path.join(folder_base, "Cropped")
folder_cropped_png = os.path.join(folder_base, "Cropped_PNG")
folder_mask_cropped = os.path.join(folder_base, "Mask_Cropped")
folder_mask_cropped_png = os.path.join(folder_base, "Mask_Cropped_PNG")
folder_temp = os.path.join(folder_base, "SHP")
for folder in [folder_out, folder_bound, folder_fishnet, folder_clipped, folder_cropped,
               folder_cropped_png, folder_mask_cropped, folder_mask_cropped_png, folder_temp]:
    os.makedirs(folder)

In [3]:
class ImageProducts:
    
    def __init__(self, image_in, name, folder_out, folder_bound, folder_fishnet, folder_clipped, folder_cropped,
                 folder_cropped_png, folder_mask_cropped, folder_mask_cropped_png, folder_temp, 
                 cell_size, pixel_size):
        
        self.image_in = image_in
        self.name = name
        self.folder_out = folder_out
        self.folder_bound = folder_bound
        self.folder_fishnet = folder_fishnet
        self.folder_clipped = folder_clipped
        self.folder_cropped = folder_cropped
        self.folder_cropped_png = folder_cropped_png
        self.folder_mask_cropped = folder_mask_cropped
        self.folder_mask_cropped_png = folder_mask_cropped_png
        self.folder_temp = folder_temp
        self.cell_size = cell_size
        self.pixel_size = pixel_size
        
        image_out = os.path.join(self.folder_out, f'{name}.tif')
        bound_out = os.path.join(self.folder_bound, f'{name}.shp')
        fishnet_out = os.path.join(self.folder_fishnet, f'{name}.shp')
        
        # resample and prepare data to create bound & fishnet
        self.Resample(self.image_in, image_out, self.pixel_size)
        epsg, shape = self.GetEPSG(image_out)

        # create bound & fishnet
        xres = shape[2]//self.cell_size*self.cell_size
        yres = shape[1]//self.cell_size*self.cell_size
        bounds = self.CreateCutline(self.GetBBOXCentroid(image_out, epsg), epsg, xres, yres, 
                                    offset_x=0, offset_y=0, out_shp=bound_out)
        net = self.CreateFishnet(bounds, epsg, cell_size, fishnet_out)
        # clip
        clipped_out = os.path.join(self.folder_clipped, f'{name}.tif')
        self.ClipImage(self.image_in, clipped_out, bound_out)
        self.ClipImageByFishnet(self.image_in, self.name, self.folder_cropped, self.folder_cropped_png, 
                                self.folder_mask_cropped, self.folder_mask_cropped_png, fishnet_out)
    
    def Resample(self, image_in, image_out, pixel_size):
        params = {'EPSG': 'UTM','COMPRESS': 'DEFLATE', 'PixelSize': pixel_size, 'NoDataValue': 0.0}
        SetRaster(rpath=image_in, tpath=image_out, miss=None, **params)
        return image_out

    def PolyFromPoints(self, xmin, xmax, ymin, ymax):
        return Polygon([(xmax, ymin), (xmax, ymax), (xmin, ymax), (xmin, ymin), (xmax, ymin)])

    def CreateGeometryGDF(self, values, epsg):
        return gpd.GeoDataFrame({'geometry': [values]}).set_crs(epsg=epsg)
    
    def GetEPSG(self, image_out):
        ds = gdal.Open(image_out)
        arr = ds.ReadAsArray()
        proj = osr.SpatialReference(wkt=ds.GetProjection())
        epsg = proj.GetAttrValue('AUTHORITY', 1)
        shape = arr.shape
        ds = None
        return epsg, shape

    def GetImageBoundaryBox(self, image_ds):
        image_geotransform = image_ds.GetGeoTransform()
        if image_geotransform != (0.0, 1.0, 0.0, 0.0, 0.0, 1.0):
            # get resolution
            xsize = image_ds.RasterXSize
            ysize = image_ds.RasterYSize
            # get coordinates of boundary box
            xmin = image_geotransform[0]
            ymin = image_geotransform[3] + xsize * image_geotransform[4] + ysize * image_geotransform[5]
            xmax = image_geotransform[0] + xsize * image_geotransform[1] + ysize * image_geotransform[2]
            ymax = image_geotransform[3]
            return self.PolyFromPoints(xmin, xmax, ymin, ymax)

    def GetBBOXCentroid(self, image, epsg):
        ds = gdal.Open(image, 0)
        geom_bbox = self.GetImageBoundaryBox(ds)
        ds = None
        gdf = self.CreateGeometryGDF(geom_bbox, epsg)
        centr = gdf.centroid
        return centr.to_crs(f"EPSG:{epsg}")

    def CreateCutline(self, centr, epsg, xres, yres, offset_x, offset_y, out_shp=None):    
        xmin = round(list(centr.x)[0]) - xres + offset_x
        xmax = round(list(centr.x)[0]) + xres + offset_x
        ymin = round(list(centr.y)[0]) - yres + offset_y
        ymax = round(list(centr.y)[0]) + yres + offset_y
        cut_bbox = self.PolyFromPoints(xmin, xmax, ymin, ymax)
        gdf = self.CreateGeometryGDF(cut_bbox, epsg)
        if out_shp is not None:
            gdf.to_file(out_shp)
        return gdf

    def CreateFishnet(self, bounds, epsg, cell_size, out_shp=None):
        xmin, ymin, xmax, ymax = bounds.total_bounds
        # create geometry of fishnet
        x_corn = list(range(int(xmin), int(xmax+0.5*cell_size), int(0.5*cell_size)))
        x_corn_list = []
        for i in range(len(x_corn)):
            if i < len(x_corn) - 2:
                x_corn_list.append([x_corn[i], x_corn[i+2]])
        y_corn = list(range(int(ymin), int(ymax+0.5*cell_size), int(0.5*cell_size)))
        y_corn_list = []
        for i in range(len(y_corn)):
            if i < len(y_corn) - 2:
                y_corn_list.append([y_corn[i], y_corn[i+2]])
        geom_array = []
        for y_corner in y_corn_list:
            for x_corner in x_corn_list:
                geom = self.PolyFromPoints(x_corner[0], x_corner[1], y_corner[0], y_corner[1])
                geom_array.append(str(geom))
        # create gdf
        df = pd.DataFrame({'wkt': geom_array})
        gs = gpd.GeoSeries.from_wkt(df['wkt'])
        gdf = gpd.GeoDataFrame(df, geometry=gs).set_crs(epsg=epsg)
        gdf_res = gdf.drop(columns=['wkt'])
        # export result
        if out_shp is not None:
            gdf_res.to_file(out_shp)
            if len(gdf_res) == 0:
                print(f'Bad extent for {out_shp}')
        return gdf_res

    def ClipImage(self, image_in, clipped_out, bound):
        params = {'EPSG': 'UTM','COMPRESS': 'DEFLATE', 'PixelSize': pixel_size, 'NoDataValue': 0.0,
                  '__vector_clipper': VectorClipper(bound, remove_pits_state = 'YES')}
        SetRaster(rpath=image_in, tpath=clipped_out, miss=None, **params)
        return image_in

    def TIFJPG(self, image, image_png): 
        gdal.Translate(destName=image_png, srcDS=image, outputType=gdalconst.GDT_Byte)

    def ClipImageByFishnet(self, image_in, name, folder_cropped, folder_cropped_png, folder_mask_cropped, 
                           folder_mask_cropped_png, fishnet_out):
        gdf = gpd.read_file(fishnet_out)
        for i in range(len(gdf)):
            cell = gdf[i:i+1]
            cell_file = os.path.join(folder_temp, f'{name}_{i}.shp')
            cropped_out = os.path.join(folder_cropped, f'{name}_{i}.tif')
            cropped_out_png = os.path.join(folder_cropped_png, f'{name}_{i}.png')
            mask_cropped_out = os.path.join(folder_mask_cropped, f'{name}_{i}.tif')
            mask_cropped_out_png = os.path.join(folder_mask_cropped_png, f'{name}_{i}.png')
            cell.to_file(cell_file)
            self.ClipImage(image_in, cropped_out, cell_file)
            try:
                self.TIFJPG(cropped_out, cropped_out_png)
            except:
                pass
            try:
                ds = gdal.Open(cropped_out)
                arr = ds.ReadAsArray()
                data = arr*0
                gdal_array.SaveArray(data.astype("byte"), mask_cropped_out, "GTIFF", ds)
                ds = None
                self.TIFJPG(mask_cropped_out, mask_cropped_out_png)
            except:
                pass

for image_in in sorted(Files(folder_in)):
    cell_size = 512.0
    pixel_size = 2
    name = SplitPath(image_in)[1]
    ImageProducts(image_in, name, folder_out, folder_bound, folder_fishnet, folder_clipped, folder_cropped,
                 folder_cropped_png, folder_mask_cropped, folder_mask_cropped_png, folder_temp, cell_size, pixel_size)

making_raster_cover
1
MULTIPOLYGON (((652291.878597818 6654729.99425363,653627.342861912 6654785.40567994,653684.718200894  |  84.718200894 6653408.54531774,652348.754799169 6653353.12009157,652291.878597818 6654729.99425363)))
making_raster_cover
1
MULTIPOLYGON (((652291.878597818 6654729.99425363,653627.342861912 6654785.40567994,653684.718200894  |  84.718200894 6653408.54531774,652348.754799169 6653353.12009157,652291.878597818 6654729.99425363)))
making_raster_cover
1
MULTIPOLYGON (((652291.878597818 6654729.99425363,653627.342861912 6654785.40567994,653684.718200894  |  84.718200894 6653408.54531774,652348.754799169 6653353.12009157,652291.878597818 6654729.99425363)))
making_raster_cover
1
MULTIPOLYGON (((652291.878597818 6654729.99425363,653627.342861912 6654785.40567994,653684.718200894  |  84.718200894 6653408.54531774,652348.754799169 6653353.12009157,652291.878597818 6654729.99425363)))
making_raster_cover
1
MULTIPOLYGON (((652291.878597818 6654729.99425363,653627.342861912

making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002

making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002

making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002

making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002 1971649.91538467,809334.2710108 1  |  9334.2710108 1968635.36897933,805408.088733474 1968574.64433853,805361.729009235 1971589.11007164)))
making_raster_cover
1
MULTIPOLYGON (((805361.729009235 1971589.11007164,809287.313429002

...

making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733

1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,10

1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,10

1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,10

1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,1015939.54519458  |  5939.54519458 3393085.43748254,1010980.9781212 3392849.32788128,1010777.05150933 3397150.42710631)))
making_raster_cover
1
MULTIPOLYGON (((1010777.05150933 3397150.42710631,1015733.62237733 3397386.71179887,10